In [ ]:
# Unified Polygon Analysis
import osmnx as ox
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString
from shapely.ops import polygonize
import numpy as np
from tqdm import tqdm
import scipy.stats
import pandas as pd
from IPython.display import display
import matplotlib.lines as mlines

# Define the polygon for Brussels
Venice_Polygon = Polygon([
    (12.367753086504578, 45.42439876839208),
    (12.367934955397544, 45.43065300973533),
    (12.361933281929645, 45.437034174341065),
    (12.364428398061813, 45.43846482392076),
    (12.363335951737222, 45.44229740469973),
    (12.351925956791497, 45.439997887472806),
    (12.328013520505165, 45.45064301329435),
    (12.315146930459987, 45.44842899261754),
    (12.29932680063853, 45.43387605442983),
    (12.315331263535567, 45.43055775303852),
    (12.334427497297073, 45.42762217007549),
    (12.347340188697714, 45.43183404583614),
    (12.360434748991318, 45.42302703806107),
    (12.367753086504578, 45.42439876839208),
])

# Initialize storage variables
phi_values = []
all_face_areas = {}
phi_statistics = {}
city = "Sofia"

# Extract road network from the polygon
G = ox.graph_from_polygon(Venice_Polygon, network_type="walk")
print("--------- City:", city)

# Plot the road network
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_facecolor('black')
ox.plot_graph(G, ax=ax, node_size=0, edge_color='gray', edge_linewidth=0.5, show=False)
plt.title(f"Road Network of {city}", color='white')
plt.show()

# Convert graph to undirected and extract edges
G_undirected = nx.Graph(G)
gdf_edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
gdf_edges = gdf_edges.to_crs(epsg=3857)  # Convert to metric projection

# Create a list of LineStrings representing the street edges
edge_lines = [LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])]) for u, v in G_undirected.edges()]

# Extract the enclosed faces using polygonization
faces = list(polygonize(edge_lines))
faces = gpd.GeoSeries(faces, crs='EPSG:4326').to_crs(epsg=3857)  # Convert polygons to metric projection

city_phi_values = []  # Store Phi values for the current city
face_areas = []  # Store block areas for the current city

print(f"Number of faces in {city}: {len(faces)}")

# Process each polygon
for polygon in tqdm(faces, desc=f"Processing faces for {city}"):
    A = polygon.area  # Block area
    face_areas.append(A / 10000)  # Convert m² to hectares

    # Compute the largest internal distance (D)
    coords = list(polygon.exterior.coords)
    max_dist = 0
    for i, p1 in enumerate(coords):
        for p2 in coords[i+1:]:
            dist = np.linalg.norm(np.array(p1) - np.array(p2))
            max_dist = max(max_dist, dist)
    D = max_dist  # Maximum distance

    if D > 0:
        phi = 4 * A / (np.pi * D**2)
        city_phi_values.append(phi)

# Calculate statistics
phi_mean = np.mean(city_phi_values) if city_phi_values else 0
phi_median = np.median(city_phi_values) if city_phi_values else 0
phi_std = np.std(city_phi_values, ddof=1) if len(city_phi_values) > 1 else 0
phi_statistics[city] = {'mean': phi_mean, 'median': phi_median, 'std': phi_std}
all_face_areas[city] = face_areas

# Print basic statistics
print(f"{city}: Mean Phi = {phi_mean:.3f}, Phi Std = {phi_std:.3f}, Phi Median = {phi_median:.3f}")

# Calculate and print mode
rounded_values = np.round(city_phi_values, decimals=3)
mode_result = scipy.stats.mode(rounded_values)
mode_value = float(mode_result.mode)
print(f"{city}: Phi Mode = {mode_value:.3f}")

# Calculate and print IQR and MAD
phi_array = np.array(city_phi_values)
q1 = np.percentile(phi_array, 25)
q3 = np.percentile(phi_array, 75)
iqr = q3 - q1
mad = np.mean(np.abs(phi_array - np.mean(phi_array)))
print(f"\n{city} additional statistics:")
print(f"IQR = {iqr:.3f}")
print(f"MAD = {mad:.3f}")

# Display statistics as DataFrame
phi_df = pd.DataFrame.from_dict(phi_statistics, orient='index')
display(phi_df)

# Plot histogram of face areas
plt.figure(figsize=(8, 5))
plt.hist(face_areas, bins=30, color='skyblue', edgecolor='black')
plt.xlabel("Face Area (hectares)")
plt.ylabel("Frequency")
plt.title(f"Histogram of Face Areas in {city}")
plt.show()

# Plot distribution of phi P(phi)
plt.figure(figsize=(10, 6))
bins = np.linspace(0, 1, 40)
plt.hist(city_phi_values, bins=bins, color='orange', edgecolor='black', density=True)
plt.xlabel(r"$\Phi$")
plt.ylabel(r"Probability Density $P(\Phi)$")
plt.title(f"Distribution of $\Phi$ in {city}")
plt.show()

# Box plot for phi distribution
plt.figure(figsize=(10, 6))
boxplot = plt.boxplot([city_phi_values], labels=[city], patch_artist=True, showmeans=True, meanline=True)

# Adjust line width for median and mean
line_width = 2.5
for line in boxplot['medians']:
    line.set(linewidth=line_width, color='blue')  # Set the median line width
for line in boxplot['means']:
    line.set(linewidth=line_width, color='red')   # Set the mean line width

plt.ylabel(r"$Φ = 4A / (πD^2)$")
plt.title(f"Distribution of Φ in {city}")
plt.grid(True, linestyle='--', alpha=0.6)

# Adding legend for mean and median
mean_line = mlines.Line2D([], [], color='red', linestyle='-', label='Mean', linewidth=line_width)
median_line = mlines.Line2D([], [], color='blue', linestyle='-', label='Median', linewidth=line_width)
plt.legend(handles=[mean_line, median_line], loc='upper right')
plt.show()

# Print face area summary statistics
print("\n=== FACE AREA STATISTICS (square meters) ===")
areas_m2 = np.array(face_areas) * 10000
mean_area = np.mean(areas_m2)
median_area = np.median(areas_m2)
std_area = np.std(areas_m2)
mad_area = np.mean(np.abs(areas_m2 - mean_area))
q1_area = np.percentile(areas_m2, 25)
q3_area = np.percentile(areas_m2, 75)
iqr_area = q3_area - q1_area
rounded_areas = np.round(areas_m2, -2)
mode_result = scipy.stats.mode(rounded_areas)
mode_area = float(mode_result.mode)

print(f"\n{city} face area statistics:")
print(f"Number of faces: {len(areas_m2)}")
print(f"Mode = {mode_area:.1f} m²")
print(f"Mean = {mean_area:.1f} m²")
print(f"Median = {median_area:.1f} m²")
print(f"Std = {std_area:.1f} m²")
print(f"MAD = {mad_area:.1f} m²")
print(f"IQR = {iqr_area:.1f} m²")